In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Bidirectional, LSTM, Dropout, Embedding, GRU, TimeDistributed
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import json
import pickle
import os
import argparse
from sklearn.model_selection import train_test_split
# from keras_contrib.layers import CRF

In [ ]:
doccano_label = []
for line in open('../input/project2/neu.jsonl', 'r'):
    doccano_label.append(json.loads(line))

In [ ]:
doccano_label[0]['annotations']

In [ ]:
df = pd.DataFrame(columns=['text','start_offset1', 'end_offset1','valueSao',
                          'start_offset2', 'end_offset2','valueCung',
                          'start_offset3', 'end_offset3','valueChi',
                          'start_offset4', 'end_offset4','valueCan'])

In [ ]:
for i in range(len(doccano_label)):
    
    text = doccano_label[i]['text']
    start_offset1 = []
    start_offset2 = []
    start_offset3 = []
    start_offset4 = []
    
    end_offset1 = []
    end_offset2 = []
    end_offset3 = []
    end_offset4 = []
    
    valueSao = []
    valueCung = []
    valueChi = []
    valueCan = []
    
    for j in range(len(doccano_label[i]['annotations'])):
        if doccano_label[i]['annotations'][j]['label'] == 7:
            start1 = int(doccano_label[i]['annotations'][j]['start_offset'])
            end1 = int(doccano_label[i]['annotations'][j]['end_offset'])
            start_offset1.append(start1)
            end_offset1.append(end1)
            valueSao.append(doccano_label[i]['text'][start1:end1])
        elif doccano_label[i]['annotations'][j]['label'] == 8:
            start2 = int(doccano_label[i]['annotations'][j]['start_offset'])
            end2 = int(doccano_label[i]['annotations'][j]['end_offset'])
            start_offset2.append(start2)
            end_offset2.append(end2)
            valueCung.append(doccano_label[i]['text'][start2:end2])
        elif doccano_label[i]['annotations'][j]['label'] == 9:
            start3 = int(doccano_label[i]['annotations'][j]['start_offset'])
            end3 = int(doccano_label[i]['annotations'][j]['end_offset'])
            start_offset3.append(start3)
            end_offset3.append(end3)
            valueChi.append(doccano_label[i]['text'][start3:end3])
        else:
            start4 = int(doccano_label[i]['annotations'][j]['start_offset'])
            end4 = int(doccano_label[i]['annotations'][j]['end_offset'])
            start_offset4.append(start4)
            end_offset4.append(end4)
            valueCan.append(doccano_label[i]['text'][start4:end4])
    start_offset1 = sorted(start_offset1)
    end_offset1 = sorted(end_offset1)
    start_offset2 = sorted(start_offset2)
    end_offset2 = sorted(end_offset2)
    start_offset3 = sorted(start_offset3)
    end_offset3 = sorted(end_offset3)
    start_offset4 = sorted(start_offset4)
    end_offset4 = sorted(end_offset4) 
    df = df.append({'text': text, 'valueSao':valueSao,'start_offset1':start_offset1,'end_offset1':end_offset1,
                   'valueCung':valueCung,'start_offset2':start_offset2,'end_offset2':end_offset2,
                   'valueChi':valueChi,'start_offset3':start_offset3,'end_offset3':end_offset3,
                   'valueCan':valueCan,'start_offset4':start_offset4,'end_offset4':end_offset4}, ignore_index=True)


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
def preprocess_texts(texts, tokenizer, max_seq_len):
    """
    Tokenize and then pad text sequences
    params:
        texts: list of string
        tokenizer: keras.preprocessing.text.Tokenizer object
        max_seq_len: int
    returns:
        np array, shape (len(texts), max_seq_len)
    """
    tokens_list = tokenizer.texts_to_sequences(texts)
    tokens_list_padded = pad_sequences(tokens_list, maxlen=max_seq_len)
    return tokens_list_padded

In [ ]:
def postprocess_predict_results(preds, tokens_list_padded, reverse_word_index, oov_token):
    """
    params:
        preds: np array, shape (n_samples, max_seq_len, 2)
        tokens_list_padded: np array, shape (n_samples, max_seq_len)
        reverse_word_index: dict, key: int (token), value: char
        oov_token: char, out of vocabulary token
    returns:
        list of tuple (text, predict):
    """
    #preds = np.squeeze(preds, axis=-1)
    #masks = np.zeros(preds.shape, dtype=int)
    #masks[preds >= 0.5] = 1
    masks = np.argmax(preds, axis=-1)
    results = []
    
    for tokens_padded, mask in zip(tokens_list_padded, masks):
        text = ''.join([reverse_word_index[token] for token in tokens_padded]).strip(oov_token)             

        predictSao = ['']
        predictCung = ['']
        predictChi = ['']
        predictCan = ['']
        
        for i, mask_i in enumerate(mask):
            if mask_i == 0:
                continue
            elif mask_i == 1:
                predictSao.append(reverse_word_index[tokens_padded[i]].strip(oov_token))
            elif mask_i == 2:
                predictSao[-1] += reverse_word_index[tokens_padded[i]].strip(oov_token)
                
            elif mask_i == 3:
                predictCung.append(reverse_word_index[tokens_padded[i]].strip(oov_token))
            elif mask_i == 4:
                predictCung[-1] += reverse_word_index[tokens_padded[i]].strip(oov_token)
                
            elif mask_i == 5:
                predictChi.append(reverse_word_index[tokens_padded[i]].strip(oov_token))
            elif mask_i == 6:
                predictChi[-1] += reverse_word_index[tokens_padded[i]].strip(oov_token)
                
            elif mask_i == 7:
                predictCan.append(reverse_word_index[tokens_padded[i]].strip(oov_token))
            else:
                predictCan[-1] += reverse_word_index[tokens_padded[i]].strip(oov_token)
        
#         extracted_tokens = tokens_padded[mask!=0]
#         predict = ''.join([reverse_word_index[token] for token in extracted_tokens]).strip(oov_token)
        predictSao.pop(0)
        predictCung.pop(0)
        predictChi.pop(0)
        predictCan.pop(0)
        results.append((text, predictSao, predictCung, predictChi, predictCan))
        
    return results

In [ ]:
oov_token = '@'
max_seq_len = 200

In [ ]:
corpus = list(df['text'].values)

tokenizer = Tokenizer(char_level=True, lower=False, oov_token=oov_token)
tokenizer.fit_on_texts(corpus)

vocab_size = len(tokenizer.word_index)
reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}
reverse_word_index[0] = ''

X = preprocess_texts(corpus, tokenizer, max_seq_len)
X = to_categorical(X, num_classes=vocab_size + 2, dtype='int32')
Y = []


In [ ]:
for idx, document in enumerate(corpus):
        y = [0] * len(document)
        for start, end in zip(df.loc[idx, 'start_offset1'], df.loc[idx, 'end_offset1']):
            y[start] = 1
            y[start+1:end] = [2] * (end - start-1)
        for start, end in zip(df.loc[idx, 'start_offset2'], df.loc[idx, 'end_offset2']):
            y[start] = 3
            y[start+1:end] = [4] * (end - start-1)
        for start, end in zip(df.loc[idx, 'start_offset3'], df.loc[idx, 'end_offset3']):
            y[start] = 5
            y[start+1:end] = [6] * (end - start-1)
        for start, end in zip(df.loc[idx, 'start_offset4'], df.loc[idx, 'end_offset4']):
            y[start] = 7
            y[start+1:end] = [8] * (end - start-1)
        Y.append(y)

In [ ]:
Y = pad_sequences(Y, maxlen=max_seq_len)

Y = to_categorical(Y, num_classes=9, dtype='int32') #onehot encoder

In [ ]:
# Embedding layer of same length output (180 dim embedding will be generated)
inp = Input(shape=(X.shape[1], X.shape[2]))
#model = Embedding(vocab_size+2, output_dim=max_seq_len, input_length=max_seq_len)(inp)
lstm_1 = Bidirectional(LSTM(32, return_sequences=True))(inp)
lstm_2 = Bidirectional(LSTM(64, return_sequences=True))(lstm_1)
fc = Dense(128, activation='relu')(lstm_2)
dropout = Dropout(0.25)(fc)
out = Dense(9, activation='softmax')(dropout)

model = Model(inp, out)

In [ ]:
early_stop = EarlyStopping(monitor='val_categorical_accuracy', patience=35, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5, verbose=1)
optimizer = Adam(lr=1e-3)
#parallel_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X, Y, batch_size=16, epochs=15, validation_split=0.2, callbacks=[reduce_lr, early_stop])

In [ ]:
vocab_size

In [ ]:
vocab_size = len(tokenizer.word_index)
reverse_word_index = {v: k for k, v in tokenizer.word_index.items()}
reverse_word_index[0] = ''
oov_token = '@'
max_seq_len = 200
print(reverse_word_index)

In [ ]:
#from sklearn.metrics import f1_score

#pred = model.predict(X_test)
#pred1 = np.argmax(pred, axis=-1)
#f1_score(Y_test.flatten(), pred1.flatten(), average='macro')

In [ ]:
#results = postprocess_predict_results(pred, preprocess_texts(corpus, tokenizer, max_seq_len), reverse_word_index, oov_token)
#results

In [ ]:
def _data(fn):
  with open(fn, 'r') as fn:
    data = fn.readlines()
  data = [item[:-1] for item in data]  
  return data

data = _data(fn = '../input/neu-sau/neu_sau.txt')
data[:5]

In [ ]:
corpus2 = _data(fn = '../input/neu-sau/neu_sau.txt')
tokens_list_padded2 = preprocess_texts(corpus2, tokenizer, max_seq_len)
X2 = to_categorical(tokens_list_padded2, num_classes=vocab_size + 2, dtype='int32')
preds2 = model.predict(X2)
results2 = postprocess_predict_results(preds2, tokens_list_padded2, reverse_word_index, oov_token)
results2